In [45]:
import numpy as np
import pandas as pd
import pickle

In [2]:
df=pd.read_csv("Sentiment Analysis.csv")

In [3]:
df.head()

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0


In [4]:
df.isna().sum()

text     0
label    0
dtype: int64

In [5]:
df.shape

(5000, 2)

In [6]:
import string
import nltk
import re

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
sw=nltk.corpus.stopwords.words("english")
sw[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [9]:
ps=nltk.PorterStemmer()

In [10]:
def cleantext(txt):
  lst1="".join([i for i in txt if i not in string.punctuation])
  lst2=re.split('\W+',txt)
  lst3=[i for i in lst2 if i not in sw]
  lst4=[ps.stem(i) for i in lst3]
  return lst4

df["clean"]=df["text"].apply(lambda x:cleantext(x))
df.head()

,text,label,clean
0,I always wrote this series off as being a comp...,0,"[I, alway, wrote, seri, complet, stink, fest, ..."
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0,"[1st, watch, 12, 7, 2002, 3, 10, dir, steve, p..."
2,This movie was so poorly written and directed ...,0,"[thi, movi, poorli, written, direct, I, fell, ..."
3,The most interesting thing about Miryang (Secr...,1,"[the, interest, thing, miryang, secret, sunshi..."
4,"when i first read about ""berlin am meer"" i did...",0,"[first, read, berlin, meer, expect, much, thou..."


In [11]:
x=pd.DataFrame(df["clean"])
x.head()

,clean
0,"[I, alway, wrote, seri, complet, stink, fest, ..."
1,"[1st, watch, 12, 7, 2002, 3, 10, dir, steve, p..."
2,"[thi, movi, poorli, written, direct, I, fell, ..."
3,"[the, interest, thing, miryang, secret, sunshi..."
4,"[first, read, berlin, meer, expect, much, thou..."


In [12]:
y=pd.DataFrame(df["label"])
y.head()

,label
0,0
1,0
2,0
3,1
4,0


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [14]:
import tensorflow
from tensorflow import keras

In [15]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(x_train["clean"])
x_train_seq=tokenizer.texts_to_sequences(x_train["clean"])
x_test_seq=tokenizer.texts_to_sequences(x_test["clean"])

In [17]:
x_train_seq_padded=pad_sequences(x_train_seq,50)
x_test_seq_padded=pad_sequences(x_test_seq,50)

In [18]:
from keras.layers import Dense, Embedding,LSTM
from keras.models import Sequential

In [19]:
#construct basic RNN Model
model_sq=Sequential()
model_sq.add(Embedding(len(tokenizer.index_word)+1,32))
model_sq.add(LSTM(32,dropout=0,recurrent_dropout=0))
model_sq.add(Dense(32,activation='relu'))
model_sq.add(Dense(1,activation='sigmoid'))
model_sq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          787808    
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 797,217
Trainable params: 797,217
Non-trainable params: 0
_________________________________________________________________


In [20]:
#compile the model
model_sq.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
#fit the RNN model
model_sq.fit(x_train_seq_padded,y_train['label'],batch_size=32,epochs=5,
             validation_data=(x_test_seq_padded,y_test))

Epoch 1/5
125/125 [==============================] - 5s 24ms/step - loss: 0.6317 - accuracy: 0.6360 - val_loss: 0.4542 - val_accuracy: 0.8120
Epoch 2/5
125/125 [==============================] - 2s 19ms/step - loss: 0.3022 - accuracy: 0.8788 - val_loss: 0.4479 - val_accuracy: 0.8050
Epoch 3/5
125/125 [==============================] - 2s 18ms/step - loss: 0.0990 - accuracy: 0.9693 - val_loss: 0.5570 - val_accuracy: 0.7890
Epoch 4/5
125/125 [==============================] - 2s 17ms/step - loss: 0.0395 - accuracy: 0.9890 - val_loss: 0.6383 - val_accuracy: 0.7890
Epoch 5/5
125/125 [==============================] - 2s 16ms/step - loss: 0.0145 - accuracy: 0.9940 - val_loss: 0.7262 - val_accuracy: 0.7820


In [37]:
data="Hi there, I hope you are doing good! Take care and love"

In [38]:
newdata=pd.DataFrame({"text":[s]})
newdata

,text
0,"Hi there, I hope you are doing good! Take care..."


In [39]:
newdata["clean"]=newdata["text"].apply(lambda x:cleantext(x))
newdata.head()

,text,clean
0,"Hi there, I hope you are doing good! Take care...","[Hi, I, hope, good, take, care, love]"


In [40]:
tokenizer.fit_on_texts(newdata["clean"])
x=tokenizer.texts_to_sequences(newdata["clean"])

In [42]:
x_padded=pad_sequences(x,50)

In [43]:
model_sq.predict(x_padded)

1/1 [==============================] - 0s 398ms/step


array([[0.820321]], dtype=float32)

In [52]:
model_sq.save("sentiment.h5",save_format='h5')

In [56]:
model_load=tensorflow.keras.models.load_model("sentiment.h5")

In [57]:
model_load.predict(x_padded)

1/1 [==============================] - 0s 319ms/step


array([[0.820321]], dtype=float32)